In [1]:
import externals.metrics.NISQA.nisqa.NISQA_lib as NL


In [2]:
model = NL.NISQA()

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--mode', required=True, type=str, help='either predict_file, predict_dir, or predict_csv')
parser.add_argument('--pretrained_model', required=True, type=str, help='file name of pretrained model (must be in current working folder)')
parser.add_argument('--deg', type=str, help='path to speech file')
parser.add_argument('--data_dir', type=str, help='folder with speech files')
parser.add_argument('--output_dir', type=str, help='folder to ouput results.csv')
parser.add_argument('--csv_file', type=str, help='file name of csv (must be in current working folder)')
parser.add_argument('--csv_deg', type=str, help='column in csv with files name/path')
parser.add_argument('--num_workers', type=int, default=0, help='number of workers for pytorchs dataloader')
parser.add_argument('--bs', type=int, default=1, help='batch size for predicting')
parser.add_argument('--ms_channel', type=int, help='audio channel in case of stereo file')

NameError: name 'argparse' is not defined

In [6]:
!python externals/metrics/NISQA/run_predict.py  --mode predict_dir\
     --pretrained_model /home/filip/speech_metrics_eval/externals/metrics/NISQA/weights/nisqa.tar \
        --data_dir /work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev\
            --output_dir ./results/nisqa

Device: cuda
Model architecture: NISQA_DIM
Loaded pretrained model from /home/filip/speech_metrics_eval/externals/metrics/NISQA/weights/nisqa.tar
# files: 3197
---> Predicting ...
/work/miniconda3/envs/sayso/lib/python3.9/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(
     deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
3538.wav  2.852255  2.449323  3.450326  2.690929   3.584681 NISQAv2
3479.wav  1.868043  2.047994  3.470353  2.418661   2.906478 NISQAv2
1371.wav  1.848897  2.193228  3.486899  2.219808   2.558506 NISQAv2
3899.wav  1.911147  2.876427  2.661885  2.280277   2.834882 NISQAv2
0936.wav  2.172540  2.706216  4.374249  3.506800   2.761111 NISQAv2
0133.wav  2.320482  2.943306  3.387454  2.045056   2.825163 NISQAv2
1819.wav  1.860562  2.505559  3.692092  1.832742   2.090843 NISQAv2
3289.wav  

In [17]:
!cd ./externals/metrics/Noresqa/ && python main.py --GPU_id -2 --mode file --test_file '/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/3538.wav' --nmr '/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV'

Durations dont match. Adjusting duration of reference.
Probaility of the test speech cleaner than the given NMR = 0.13384871
NORESQA score of the test speech with respect to the given NMR = 15.769922626001062


In [6]:
import sys

sys.path.append("/home/filip/samsara/elysium/externals/Noresqa")

import torch
import argparse
import librosa as librosa
import torch.nn.functional as F
import numpy as np  
import torch.nn as nn
from model import NORESQA
from scipy import signal
import glob
import ntpath

model = NORESQA(output=40, output2=40)

# Loading checkpoint
model_checkpoint_path = '/home/filip/samsara/elysium/externals/Noresqa/models/model.pth'
state = torch.load(model_checkpoint_path,map_location="cpu")['state_base']

pretrained_dict = {}
for k, v in state.items():
    if 'module' in k:
        pretrained_dict[k.replace('module.','')]=v
    else:
        pretrained_dict[k]=v
model_dict = model.state_dict()
model_dict.update(pretrained_dict)
model.load_state_dict(pretrained_dict)

# device
device = torch.device("cuda:{}".format(2))

model.to(device)
model.eval()

sfmax = nn.Softmax(dim=1)

# function extraction stft
def extract_stft(audio, sampling_rate = 16000):

    f, t, Sxx = signal.stft(audio, sampling_rate, window='hann',nperseg=512,noverlap=256,nfft=512)
    Sxx = Sxx[:256,:]

    feat = np.concatenate((np.abs(Sxx).reshape([Sxx.shape[0],Sxx.shape[1],1]), np.angle(Sxx).reshape([Sxx.shape[0],Sxx.shape[1],1])), axis=2)

    return feat 

# function doing prediction
def model_prediction(test_feat, nmr_feat):

    intervals_sdr = np.arange(0.5,40,1) 

    with torch.no_grad():
        
        ranking_frame,sdr_frame,snr_frame = model(test_feat.permute(0,3,2,1),nmr_feat.permute(0,3,2,1))
        # preference task prediction
        
        ranking = sfmax(ranking_frame).mean(2).detach().cpu().numpy()
        pout = ranking[0][0]

        # quantification task
        sdr = intervals_sdr * (sfmax(sdr_frame).mean(2).detach().cpu().numpy())
        qout = sdr.sum()

    return pout, qout

# function checking if the size of the inputs are same. If not, then the reference audio's size is adjusted
def check_size(audio_ref,audio_test):
    
    if len(audio_ref) > len(audio_test):
        print('Durations dont match. Adjusting duration of reference.')
        audio_ref = audio_ref[:len(audio_test)]
        
    elif len(audio_ref) < len(audio_test):
        print('Durations dont match. Adjusting duration of reference.')
        while len(audio_test) > len(audio_ref):
            audio_ref = np.append(audio_ref, audio_ref)
        audio_ref = audio_ref[:len(audio_test)] 
    
    return audio_ref, audio_test

# reading audio clips
def audio_loading(path,sampling_rate=16000):

    audio, fs = librosa.load(path)
    if len(audio.shape) > 1:
        audio = librosa.to_mono(audio)

    if fs != sampling_rate:
        audio = librosa.resample(audio,fs,sampling_rate)

    return audio

# top level function combining loading, and feature extraction
def feats_loading(ref_path,test_path):
    
    audio_ref = audio_loading(ref_path)
    audio_test = audio_loading(test_path)

    audio_ref, audio_test = check_size(audio_ref,audio_test)

    ref_feat = extract_stft(audio_ref)
    test_feat = extract_stft(audio_test)

    return ref_feat,test_feat

def inference_file(nmr,test_file):
    nmr_feat,test_feat = feats_loading(nmr,test_file)
    test_feat = torch.from_numpy(test_feat).float().to(device).unsqueeze(0)
    nmr_feat = torch.from_numpy(nmr_feat).float().to(device).unsqueeze(0)

    return model_prediction(test_feat,nmr_feat)

In [7]:
# function extraction stft
def extract_stft(audio, sampling_rate = 16000):

    fx, tx, stft_out = signal.stft(audio, sampling_rate, window='hann',nperseg=512,noverlap=256,nfft=512)
    stft_out = stft_out[:256,:]
    feat = np.concatenate((np.abs(stft_out).reshape([stft_out.shape[0],stft_out.shape[1],1]), np.angle(stft_out).reshape([stft_out.shape[0],stft_out.shape[1],1])), axis=2)
    return feat

# noresqa and noresqa-mos prediction calls
def model_prediction_noresqa(test_feat, nmr_feat):

    intervals_sdr = np.arange(0.5,40,1)

    with torch.no_grad():
        ranking_frame,sdr_frame,snr_frame = model(test_feat.permute(0,3,2,1),nmr_feat.permute(0,3,2,1))
        # preference task prediction
        ranking = sfmax(ranking_frame).mean(2).detach().cpu().numpy()
        pout = ranking[0][0]
        # quantification task
        sdr = intervals_sdr * (sfmax(sdr_frame).mean(2).detach().cpu().numpy())
        qout = sdr.sum()

    return pout, qout

def model_prediction_noresqa_mos(test_feat, nmr_feat):

    with torch.no_grad():
        score = model(nmr_feat,test_feat).detach().cpu().numpy()[0]

    return score


# reading audio clips
def audio_loading(path,sampling_rate=16000):

    audio, fs = librosa.load(path)
    if len(audio.shape) > 1:
        audio = librosa.to_mono(audio)

    if fs != sampling_rate:
        audio = librosa.resample(audio,fs,sampling_rate)

    return audio


# function checking if the size of the inputs are same. If not, then the reference audio's size is adjusted
def check_size(audio_ref,audio_test):

    if len(audio_ref) > len(audio_test):
        print('Durations dont match. Adjusting duration of reference.')
        audio_ref = audio_ref[:len(audio_test)]

    elif len(audio_ref) < len(audio_test):
        print('Durations dont match. Adjusting duration of reference.')
        while len(audio_test) > len(audio_ref):
            audio_ref = np.append(audio_ref, audio_ref)
        audio_ref = audio_ref[:len(audio_test)]

    return audio_ref, audio_test


# audio loading and feature extraction
def feats_loading(test_path, ref_path=None, noresqa_or_noresqaMOS = 0):

    if noresqa_or_noresqaMOS == 0 or noresqa_or_noresqaMOS == 1:

        audio_ref = audio_loading(ref_path)
        audio_test = audio_loading(test_path)
        audio_ref, audio_test = check_size(audio_ref,audio_test)

        if noresqa_or_noresqaMOS == 0:
            ref_feat = extract_stft(audio_ref)
            test_feat = extract_stft(audio_test)
            return ref_feat,test_feat
        else:
            return audio_ref, audio_test

In [9]:
from externals.metrics.Noresqa.model import NORESQA
import fairseq
import torchaudio
import torch
from torch import nn
device = torch.device("cuda:{}".format(1))

In [10]:
model = NORESQA(metric_type=1,config_path="/home/filip/speech_metrics_eval/externals/metrics/Noresqa/models/wav2vec_small.pt").to(device)

/work/miniconda3/envs/sayso_dev/lib/python3.9/site-packages/hydra/core/plugins.py:202: UserWarning: 
	Error importing 'hydra_plugins.hydra_colorlog'.
	Plugin is incompatible with this Hydra version or buggy.
	Recommended to uninstall or upgrade plugin.
		ImportError : cannot import name 'SearchPathPlugin' from 'hydra.plugins' (/work/miniconda3/envs/sayso_dev/lib/python3.9/site-packages/hydra/plugins/__init__.py)
  warnings.warn(


In [18]:
noisy_audio,sr = torchaudio.load('/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV')
clean_audio,sr =torchaudio.load(glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*")[0])

In [55]:
a, b = feats_loading('/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV', 
glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*")[1], noresqa_or_noresqaMOS = 1)

Durations dont match. Adjusting duration of reference.


In [49]:
glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*")[0]

'/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/3538.wav'

In [35]:
clean_audio.unsqueeze(0).shape

torch.Size([1, 1, 100800])

In [61]:
model(torch.tensor(b).unsqueeze(0).to(device),
torch.tensor(a).unsqueeze(0).to(device))[0][0].item()

1.9326815605163574

In [62]:
inferenced_files = []
for tencent_file in glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*"):
    a, b = feats_loading('/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV', tencent_file, noresqa_or_noresqaMOS = 1)
    out = model(torch.tensor(b).unsqueeze(0).to(device), torch.tensor(a).unsqueeze(0).to(device))[0][0].item()
    inferenced_files.append((ntpath.basename(tencent_file),out))

Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations 

In [53]:
model(torch.tensor(b).unsqueeze(0).to(device),torch.tensor(a).unsqueeze(0).to(device))

(tensor([1.9042], device='cuda:1', grad_fn=<SumBackward1>),
 tensor([[0.0864, 0.0142]], device='cuda:1', grad_fn=<AddmmBackward0>))

In [11]:
inference_file('/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV',
glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*")[0])

Durations dont match. Adjusting duration of reference.


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [512, 1, 10], but got 5-dimensional input of size [1, 1, 2, 236, 256] instead

In [39]:
# inferenced_files = []
# for tencent_file in glob.glob("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDev/*"):
#     out = inference_file('/work/data/timit/data/TEST/DR1/FAKS0/SI943.WAV',tencent_file)
#     inferenced_files.append((ntpath.basename(tencent_file),out[0],out[1]))


Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations dont match. Adjusting duration of reference.
Durations 

In [ ]:
import pandas as pd
  
  
# Calling DataFrame constructor on list
df = pd.DataFrame(inferenced_files)
df

,0,1,2
0,3538.wav,0.133849,15.769922
1,3479.wav,0.487211,11.310331
2,1371.wav,0.461265,11.604966
3,3899.wav,0.202663,14.257084
4,0936.wav,0.093169,21.569554
...,...,...,...
3192,1302.wav,0.443649,13.000333
3193,2753.wav,0.485969,9.527665
3194,3639.wav,0.513667,10.831978
3195,3395.wav,0.125497,14.726045


In [ ]:
df.columns = ['deg','NORESQA_MOS']

In [ ]:
df.to_csv("/home/filip/speech_metrics_eval/results/noresqa/NORESQA_MOS_results.csv")

In [ ]:
tencent_mos = pd.read_csv("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDevMOS.csv")
tencent_mos['deg'] = tencent_mos.apply(lambda x: ntpath.basename(x['deg_wav']), axis=1)

In [ ]:
metrics_with_mos = pd.merge(df,tencent_mos, on='deg', how='inner')

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.metrics import mean_squared_error

pccs = np.corrcoef(metrics_with_mos["mos"], metrics_with_mos["NORESQA_MOS"])[0][1]
SROCC = stats.spearmanr(metrics_with_mos["mos"], metrics_with_mos["NORESQA_MOS"])[0]

print(f"NORESQA_MOS -> Pearson Correlation Coefficient: {pccs}, Spearman's RankOrder Correlation: {SROCC}")

In [42]:
df.columns = ['deg', 'cleaner_speech_prob','NORESQA_score']

In [43]:
df.to_csv("/home/filip/speech_metrics_eval/results/noresqa/NORESQA_results.csv")

In [44]:
noresqa_res = pd.read_csv("/home/filip/speech_metrics_eval/results/noresqa/NORESQA_results.csv")
nisqa_res = pd.read_csv("/home/filip/speech_metrics_eval/results/nisqa/NISQA_results.csv")

In [56]:
metrics_res = pd.merge(noresqa_res, nisqa_res, on='deg', how='inner')

In [53]:
#pd.concat([noresqa_res,nisqa_res],axis=1)

In [54]:
tencent_mos = pd.read_csv("/work/data/speech_metrics_eval/TencentCorups/withReverberationTrainDevMOS.csv")
tencent_mos['deg'] = tencent_mos.apply(lambda x: ntpath.basename(x['deg_wav']), axis=1)

In [57]:
metrics_with_mos = pd.merge(metrics_res,tencent_mos, on='deg', how='inner')

In [58]:
metrics_with_mos

,Unnamed: 0,deg,cleaner_speech_prob,NORESQA_score,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,model,deg_wav,mos
0,0,3538.wav,0.133849,15.769922,2.852255,2.449323,3.450326,2.690929,3.584681,NISQAv2,./withReverberationTrainDev/3538.wav,3.24
1,1,3479.wav,0.487211,11.310331,1.868043,2.047994,3.470353,2.418661,2.906478,NISQAv2,./withReverberationTrainDev/3479.wav,1.80
2,2,1371.wav,0.461265,11.604966,1.848897,2.193228,3.486899,2.219808,2.558506,NISQAv2,./withReverberationTrainDev/1371.wav,2.12
3,3,3899.wav,0.202663,14.257084,1.911147,2.876427,2.661885,2.280277,2.834882,NISQAv2,./withReverberationTrainDev/3899.wav,3.32
4,4,0936.wav,0.093169,21.569554,2.172540,2.706216,4.374250,3.506800,2.761111,NISQAv2,./withReverberationTrainDev/0936.wav,3.40
...,...,...,...,...,...,...,...,...,...,...,...,...
3192,3192,1302.wav,0.443649,13.000333,0.899307,3.462421,2.121909,1.965579,1.583098,NISQAv2,./withReverberationTrainDev/1302.wav,1.00
3193,3193,2753.wav,0.485969,9.527665,1.726453,2.209515,3.656949,1.938272,1.968538,NISQAv2,./withReverberationTrainDev/2753.wav,1.92
3194,3194,3639.wav,0.513667,10.831978,1.970071,1.837899,3.578102,2.662608,2.714570,NISQAv2,./withReverberationTrainDev/3639.wav,2.28
3195,3195,3395.wav,0.125497,14.726045,3.120940,2.868086,4.049940,2.826176,3.229072,NISQAv2,./withReverberationTrainDev/3395.wav,3.84


In [62]:
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.metrics import mean_squared_error

pccs = np.corrcoef(metrics_with_mos["mos"], metrics_with_mos["mos_pred"])[0][1]
SROCC = stats.spearmanr(metrics_with_mos["mos"], metrics_with_mos["mos_pred"])[0]

print(f"NISQA -> Pearson Correlation Coefficient: {pccs}, Spearman's RankOrder Correlation: {SROCC}")

NISQA -> Pearson Correlation Coefficient: 0.612970892771744, Spearman's RankOrder Correlation: 0.5736058135424721


In [63]:
pccs = np.corrcoef(metrics_with_mos["mos"], metrics_with_mos["NORESQA_score"])[0][1]
SROCC = stats.spearmanr(metrics_with_mos["mos"], metrics_with_mos["NORESQA_score"])[0]

print(f"NORESQA -> Pearson Correlation Coefficient: {pccs}, Spearman's RankOrder Correlation: {SROCC}")

NORESQA -> Pearson Correlation Coefficient: 0.06095500417961022, Spearman's RankOrder Correlation: 0.03461319645569539


In [ ]:
# retrain
# sdr and pesq

In [ ]:
import pandas as pd

pd.read_csv("/work/data/speech_metrics_eval/pstn_train/pstn_train.csv")

: 